Import

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math as math
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model
import sklearn
from sklearn import svm, preprocessing 
import warnings

Reading Data

In [2]:
data = pd.read_csv("../../data_sets/Volumetric_features.csv")

df = pd.DataFrame(data, columns = data.columns[:-1])
df = df.drop(['S.No','Age'], axis=1)

df.head()
df.describe()


scaler = StandardScaler()
x = scaler.fit_transform(df)
n = 20
pca = PCA(n_components=n)
pca_dataF = pca.fit_transform(x)

labels = df.columns.values.tolist()
label_index = [np.abs(pca.components_[i]).argmax() for i in range(n)]
columns = [labels[label_index[i]] for i in range(n)]

pca_df = pd.DataFrame(data=pca_dataF, columns=columns)
print(pca_df.head)
pca_df.head()

<bound method NDFrame.head of       rh_MeanThickness_thickness  CerebralWhiteMatterVol  \
0                       1.754401                1.293661   
1                       1.417516                1.506793   
2                       2.060537                1.356491   
3                       4.321472                1.316560   
4                       3.432616                1.645477   
...                          ...                     ...   
4221                    3.508241                2.349695   
4222                    4.445945               -2.409495   
4223                    8.016491                2.326577   
4224                   -0.596625                2.547033   
4225                   -2.307309               -3.481648   

      Left-Lateral-Ventricle  lh_pericalcarine_thickness  SurfaceHoles  \
0                   1.399974                   -1.478197     -1.848633   
1                   1.653943                   -0.613219     -1.385475   
2                   1.48901

,rh_MeanThickness_thickness,CerebralWhiteMatterVol,Left-Lateral-Ventricle,lh_pericalcarine_thickness,SurfaceHoles,CC_Posterior,rh_caudalanteriorcingulate_thickness,CC_Posterior,Right-Caudate,lh_parahippocampal_thickness,MaskVol-to-eTIV,Brain-Stem,Left-vessel,Right-vessel,non-WM-hypointensities,rh_isthmuscingulate_thickness,5th-Ventricle,5th-Ventricle,5th-Ventricle,lh_temporalpole_thickness
0,1.754401,1.293661,1.399974,-1.478197,-1.848633,2.545115,-1.129790,-0.415227,-0.355843,1.563833,-0.209619,0.222596,-1.506058,0.623235,-0.404856,0.634333,-0.128379,-0.557206,0.083650,0.648750
1,1.417516,1.506793,1.653943,-0.613219,-1.385475,2.367465,-1.431912,-0.859137,-0.756735,1.724350,0.197748,0.432861,-1.373817,0.260384,-0.814589,0.302409,0.440995,-0.531165,0.595680,0.445524
2,2.060537,1.356491,1.489019,-1.054750,-1.176025,2.167326,-2.149564,-0.821279,-1.122506,1.476087,0.087310,0.420047,-0.824814,0.758348,-0.787243,0.582079,0.592741,-0.148105,0.602551,0.611462
3,4.321472,1.316560,1.100123,-0.239447,-1.375439,2.419048,-1.135515,-1.568896,-0.825107,1.328358,0.306643,-0.272177,-1.290505,0.610609,-0.225332,0.290866,0.189412,-0.397606,0.581832,0.805594
4,3.432616,1.645477,2.126552,-0.803511,-1.323038,3.075424,-1.197125,-0.823745,-0.526088,1.533264,-0.093314,0.458286,-1.328653,1.274577,-0.778024,0.163895,-0.146358,-0.671014,0.579409,0.702730


Training Data

In [6]:
# Split for validation --> train, val, test = 80/15/5

#x_train, x_validation, y_train, y_validation = train_test_split(data.drop(['S.No','Age'], axis=1), data['Age'], test_size=0.20, random_state=33)

x_train, x_validation, y_train, y_validation = train_test_split(pca_df, data['Age'], test_size=0.20, random_state=33)
# (20% of total dataset -> 75% validation = 15% total, 25% validation = 5% total

x_val, x_test, y_val, y_test = train_test_split(x_validation, y_validation, test_size=0.25, random_state=33)

print("x_train shape is:",x_train.shape)
print("y_train shape is:",y_train.shape, "\n")
print("x_val shape is:",x_val.shape)
print("y_val shape is:",y_val.shape, "\n")
print("x_test shape is:",x_test.shape)
print("y_test shape is:",y_test.shape)

x_train shape is: (3380, 20)
y_train shape is: (3380,) 

x_val shape is: (634, 20)
y_val shape is: (634,) 

x_test shape is: (212, 20)
y_test shape is: (212,)


SV Regression (SVM)

In [7]:
svr = svm.SVR()
svr.fit(x_train,y_train)
y_pred = svr.predict(x_test)
print("accuracy: "+ str(svr.score(x_test,y_test)*100) + "%")
print("Mean absolute error: {}".format(mean_absolute_error(y_test,y_pred)))
print("Mean squared error: {}".format(mean_squared_error(y_test,y_pred)))
print("Root mean squared error: {}".format(math.sqrt(mean_squared_error(y_test,y_pred))))
R2 = r2_score(y_test,y_pred)
print('R Squared: {}'.format(R2))
n=x_test.shape[0]
p=x_test.shape[1] - 1

adj_rsquared = 1 - (1 - R2) * ((n - 1)/(n-p-1))
print('Adjusted R Squared: {}'.format(adj_rsquared))

accuracy: 83.70409246608361%
Mean absolute error: 6.46180795834291
Mean squared error: 63.89441732274514
Root mean squared error: 7.993398358817427
R Squared: 0.8370409246608361
Adjusted R Squared: 0.8209147661637314
